## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,0,Esperance,AU,-33.8667,121.9000,55.24,79,5,20.42,2021-07-26 01:20:20,clear sky
1,1,Qaanaaq,GL,77.4840,-69.3632,39.97,68,100,0.65,2021-07-26 01:24:58,overcast clouds
2,2,Tiksi,RU,71.6872,128.8694,51.40,90,99,5.91,2021-07-26 01:24:58,light rain
3,3,Mahebourg,MU,-20.4081,57.7000,70.09,64,75,10.36,2021-07-26 01:24:58,broken clouds
4,4,Airai,TL,-8.9266,125.4092,71.62,31,70,2.98,2021-07-26 01:24:58,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
6,6,Labuhan,ID,-6.8844,112.2051,77.97,69,4,6.13,2021-07-26 01:24:59,clear sky
8,8,Georgetown,MY,5.4112,100.3354,85.93,83,20,1.01,2021-07-26 01:24:59,haze
10,10,Hilo,US,19.7297,-155.0900,78.53,89,90,8.05,2021-07-26 01:25:00,light rain
11,11,Bethel,US,41.3712,-73.4140,75.49,89,0,2.98,2021-07-26 01:24:16,clear sky
12,12,Mareeba,AU,-17.0000,145.4333,76.48,56,35,5.75,2021-07-26 01:25:00,scattered clouds
13,13,Lohardaga,IN,23.4333,84.6833,76.86,86,100,5.86,2021-07-26 01:25:00,overcast clouds
14,14,Sorong,ID,-0.8833,131.2500,81.82,72,100,10.45,2021-07-26 01:25:01,overcast clouds
18,18,Dingle,PH,10.9995,122.6711,85.23,71,91,15.30,2021-07-26 01:25:01,overcast clouds
22,22,Murgab,TM,37.4966,61.9714,80.47,39,0,9.22,2021-07-26 01:25:03,clear sky
23,23,Vaini,TO,-21.2000,-175.2000,80.76,83,20,9.22,2021-07-26 01:25:03,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
#len(preferred_cities_df)

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Date                   245
Current Description    245
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Labuhan,ID,77.97,clear sky,-6.8844,112.2051,
8,Georgetown,MY,85.93,haze,5.4112,100.3354,
10,Hilo,US,78.53,light rain,19.7297,-155.0900,
11,Bethel,US,75.49,clear sky,41.3712,-73.4140,
12,Mareeba,AU,76.48,scattered clouds,-17.0000,145.4333,
13,Lohardaga,IN,76.86,overcast clouds,23.4333,84.6833,
14,Sorong,ID,81.82,overcast clouds,-0.8833,131.2500,
18,Dingle,PH,85.23,overcast clouds,10.9995,122.6711,
22,Murgab,TM,80.47,clear sky,37.4966,61.9714,
23,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Labuhan,ID,77.97,clear sky,-6.8844,112.2051,PT PPI Lamongan Tambak C
8,Georgetown,MY,85.93,haze,5.4112,100.3354,Cititel Penang
10,Hilo,US,78.53,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Bethel,US,75.49,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
12,Mareeba,AU,76.48,scattered clouds,-17.0000,145.4333,Trinity Plains Tourist Park
13,Lohardaga,IN,76.86,overcast clouds,23.4333,84.6833,Shorya Sweet Palace
14,Sorong,ID,81.82,overcast clouds,-0.8833,131.2500,waigo splash hotel
18,Dingle,PH,85.23,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
23,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
24,Pacific Grove,US,83.01,scattered clouds,36.6177,-121.9166,Lovers Point Inn


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))